In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 1500
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 1500
N_FEATURES = 8

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 120, 16

In [12]:
def step_decay(epoch):
   initial_lrate = 1e-4
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [13]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [14]:
import os
path='/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/'

In [15]:
for i in range(1,37):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 10, 150
    n_depth=8
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(9,1), padding='same', kernel_initializer="he_normal",strides=(3,1), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(MaxPooling2D(pool_size=(4,1),strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Conv2D(filters=64, kernel_size=(5,1),padding="same",kernel_initializer="he_normal",strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2093))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='tanh'))
    model.add(Dense(n_outputs, activation='softmax'))
#     model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal",strides=3,kernel_regularizer=l1(1e-04)), \
#                                input_shape=(n_steps,n_length,n_depth)))
#     model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
#     model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
#     model.add(TimeDistributed(Activation('tanh')))
#     model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
#     model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
#     model.add(TimeDistributed(Activation('tanh')))
#     model.add(TimeDistributed(Dropout(0.2093)))
#     model.add(TimeDistributed(Activation('relu')))
#     model.add(Flatten())
#     model.add(Dense(512, activation='tanh'))
#     model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
#     model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=2.17e-3, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
#     print(test_acc)
    test_mean=statistics.mean(test_acc)
    print(test_mean)

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 27000

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 9000

x_train shape:  (17, 1500, 8)
17 training samples
y_train shape:  (17,)
num_time_periods 1500
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
x_train shape: (1500, 8)
input_shape: (1500, 8)
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]
New y_train shape:  (17, 6)
x_test shape:  (5, 1500, 8)
5 testing samples
y_test shape:  (5,)
x_train

2/2 [==============================] - 0s 50ms/step - loss: 1.7882 - accuracy: 0.1593 - val_loss: 1.7752 - val_accuracy: 0.2000

Epoch 00028: val_accuracy did not improve from 0.20000
Epoch 29/120
2/2 [==============================] - 0s 55ms/step - loss: 1.7874 - accuracy: 0.1801 - val_loss: 1.7733 - val_accuracy: 0.2000

Epoch 00029: val_accuracy did not improve from 0.20000
Epoch 30/120
2/2 [==============================] - 0s 55ms/step - loss: 1.7857 - accuracy: 0.1801 - val_loss: 1.7714 - val_accuracy: 0.2000

Epoch 00030: val_accuracy did not improve from 0.20000
Epoch 31/120
2/2 [==============================] - 0s 57ms/step - loss: 1.7847 - accuracy: 0.1801 - val_loss: 1.7710 - val_accuracy: 0.2000

Epoch 00031: val_accuracy did not improve from 0.20000
Epoch 32/120
2/2 [==============================] - 0s 49ms/step - loss: 1.7897 - accuracy: 0.1593 - val_loss: 1.7716 - val_accuracy: 0.2000

Epoch 00032: val_accuracy did not improve from 0.20000
Epoch 33/120
2/2 [==========


Epoch 00069: val_accuracy did not improve from 0.20000
Epoch 70/120
2/2 [==============================] - 0s 56ms/step - loss: 1.7962 - accuracy: 0.1593 - val_loss: 1.7708 - val_accuracy: 0.2000

Epoch 00070: val_accuracy did not improve from 0.20000
Epoch 71/120
2/2 [==============================] - 0s 152ms/step - loss: 1.7871 - accuracy: 0.1801 - val_loss: 1.7706 - val_accuracy: 0.2000

Epoch 00071: val_accuracy did not improve from 0.20000
Epoch 72/120
2/2 [==============================] - 0s 55ms/step - loss: 1.7874 - accuracy: 0.1801 - val_loss: 1.7703 - val_accuracy: 0.2000

Epoch 00072: val_accuracy did not improve from 0.20000
Epoch 73/120
2/2 [==============================] - 0s 45ms/step - loss: 1.7867 - accuracy: 0.1801 - val_loss: 1.7701 - val_accuracy: 0.2000

Epoch 00073: val_accuracy did not improve from 0.20000
Epoch 74/120
2/2 [==============================] - 0s 49ms/step - loss: 1.7865 - accuracy: 0.1801 - val_loss: 1.7698 - val_accuracy: 0.2000

Epoch 00074: 

2/2 [==============================] - 0s 34ms/step - loss: 1.7882 - accuracy: 0.1801 - val_loss: 1.7698 - val_accuracy: 0.2000

Epoch 00111: val_accuracy did not improve from 0.20000
Epoch 112/120
2/2 [==============================] - 0s 156ms/step - loss: 1.7867 - accuracy: 0.1801 - val_loss: 1.7698 - val_accuracy: 0.2000

Epoch 00112: val_accuracy did not improve from 0.20000
Epoch 113/120
2/2 [==============================] - 0s 50ms/step - loss: 1.7900 - accuracy: 0.1801 - val_loss: 1.7698 - val_accuracy: 0.2000

Epoch 00113: val_accuracy did not improve from 0.20000
Epoch 114/120
2/2 [==============================] - 0s 64ms/step - loss: 1.7907 - accuracy: 0.1801 - val_loss: 1.7698 - val_accuracy: 0.2000

Epoch 00114: val_accuracy did not improve from 0.20000
Epoch 115/120
2/2 [==============================] - 0s 40ms/step - loss: 1.7909 - accuracy: 0.1801 - val_loss: 1.7698 - val_accuracy: 0.2000

Epoch 00115: val_accuracy did not improve from 0.20000
Epoch 116/120
2/2 [====

2/2 [==============================] - 0s 42ms/step - loss: 1.7848 - accuracy: 0.1801 - val_loss: 1.7801 - val_accuracy: 0.2000

Epoch 00016: val_accuracy did not improve from 0.40000
Epoch 17/120
2/2 [==============================] - 0s 45ms/step - loss: 1.7841 - accuracy: 0.1801 - val_loss: 1.7799 - val_accuracy: 0.2000

Epoch 00017: val_accuracy did not improve from 0.40000
Epoch 18/120
2/2 [==============================] - 0s 47ms/step - loss: 1.7846 - accuracy: 0.1801 - val_loss: 1.7796 - val_accuracy: 0.2000

Epoch 00018: val_accuracy did not improve from 0.40000
Epoch 19/120
2/2 [==============================] - 0s 48ms/step - loss: 1.7844 - accuracy: 0.1801 - val_loss: 1.7793 - val_accuracy: 0.2000

Epoch 00019: val_accuracy did not improve from 0.40000
Epoch 20/120
2/2 [==============================] - 0s 42ms/step - loss: 1.7843 - accuracy: 0.1801 - val_loss: 1.7788 - val_accuracy: 0.2000

Epoch 00020: val_accuracy did not improve from 0.40000
Epoch 21/120
2/2 [==========

...


Epoch 00091: val_accuracy did not improve from 0.40000
Epoch 92/120
2/2 [==============================] - 0s 51ms/step - loss: 1.7813 - accuracy: 0.1593 - val_loss: 1.7415 - val_accuracy: 0.2000

Epoch 00092: val_accuracy did not improve from 0.40000
Epoch 93/120
2/2 [==============================] - 0s 48ms/step - loss: 1.7810 - accuracy: 0.1593 - val_loss: 1.7415 - val_accuracy: 0.2000

Epoch 00093: val_accuracy did not improve from 0.40000
Epoch 94/120
2/2 [==============================] - 0s 41ms/step - loss: 1.7783 - accuracy: 0.1801 - val_loss: 1.7415 - val_accuracy: 0.2000

Epoch 00094: val_accuracy did not improve from 0.40000
Epoch 95/120
2/2 [==============================] - 0s 45ms/step - loss: 1.7811 - accuracy: 0.1593 - val_loss: 1.7415 - val_accuracy: 0.2000

Epoch 00095: val_accuracy did not improve from 0.40000
Epoch 96/120
2/2 [==============================] - 0s 53ms/step - loss: 1.7747 - accuracy: 0.1801 - val_loss: 1.7415 - val_accuracy: 0.2000

Epoch 00096: v

Epoch 1/120
2/2 [==============================] - 0s 151ms/step - loss: 1.7919 - accuracy: 0.0600 - val_loss: 1.7911 - val_accuracy: 0.2000

Epoch 00001: val_accuracy improved from -inf to 0.20000, saving model to /media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/checkpoint.hdf5
Epoch 2/120
2/2 [==============================] - 0s 41ms/step - loss: 1.7915 - accuracy: 0.1801 - val_loss: 1.7910 - val_accuracy: 0.2000

Epoch 00002: val_accuracy did not improve from 0.20000
Epoch 3/120
2/2 [==============================] - 0s 42ms/step - loss: 1.7914 - accuracy: 0.1801 - val_loss: 1.7913 - val_accuracy: 0.2000

Epoch 00003: val_accuracy did not improve from 0.20000
Epoch 4/120
2/2 [==============================] - 0s 44ms/step - loss: 1.7913 - accuracy: 0.1801 - val_loss: 1.7916 - val_accuracy: 0.2000

Epoch 00004: val_accuracy did not improve from 0.20000
Epoch 5/120
2/2 [==============================] - 0s 48ms/step - loss: 1.7915 - accuracy: 0.1801 - val_loss: 1.792


Epoch 00041: val_accuracy did not improve from 0.40000
Epoch 42/120
2/2 [==============================] - 0s 48ms/step - loss: 1.7863 - accuracy: 0.1801 - val_loss: 1.7765 - val_accuracy: 0.2000

Epoch 00042: val_accuracy did not improve from 0.40000
Epoch 43/120
2/2 [==============================] - 0s 53ms/step - loss: 1.7867 - accuracy: 0.1801 - val_loss: 1.7763 - val_accuracy: 0.2000

Epoch 00043: val_accuracy did not improve from 0.40000
Epoch 44/120
2/2 [==============================] - 0s 51ms/step - loss: 1.7871 - accuracy: 0.1593 - val_loss: 1.7760 - val_accuracy: 0.2000

Epoch 00044: val_accuracy did not improve from 0.40000
Epoch 45/120
2/2 [==============================] - 0s 49ms/step - loss: 1.7868 - accuracy: 0.1593 - val_loss: 1.7756 - val_accuracy: 0.2000

Epoch 00045: val_accuracy did not improve from 0.40000
Epoch 46/120
2/2 [==============================] - 0s 46ms/step - loss: 1.7866 - accuracy: 0.1593 - val_loss: 1.7752 - val_accuracy: 0.2000

Epoch 00046: v

2/2 [==============================] - 0s 50ms/step - loss: 1.7855 - accuracy: 0.1801 - val_loss: 1.7664 - val_accuracy: 0.2000

Epoch 00083: val_accuracy did not improve from 0.40000
Epoch 84/120
2/2 [==============================] - 0s 54ms/step - loss: 1.7849 - accuracy: 0.1801 - val_loss: 1.7664 - val_accuracy: 0.2000

Epoch 00084: val_accuracy did not improve from 0.40000
Epoch 85/120
2/2 [==============================] - 0s 52ms/step - loss: 1.7832 - accuracy: 0.1801 - val_loss: 1.7664 - val_accuracy: 0.2000

Epoch 00085: val_accuracy did not improve from 0.40000
Epoch 86/120
2/2 [==============================] - 0s 52ms/step - loss: 1.7846 - accuracy: 0.1801 - val_loss: 1.7664 - val_accuracy: 0.2000

Epoch 00086: val_accuracy did not improve from 0.40000
Epoch 87/120
2/2 [==============================] - 0s 48ms/step - loss: 1.7850 - accuracy: 0.1801 - val_loss: 1.7664 - val_accuracy: 0.2000

Epoch 00087: val_accuracy did not improve from 0.40000
Epoch 88/120
2/2 [==========

KeyboardInterrupt: 

In [ ]:
statistics.mean(test_acc)

In [ ]:
# model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal",strides=3,kernel_regularizer=l1(1e-04)), \
#                            input_shape=(n_steps,n_length,n_depth)))
# model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
# model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
# model.add(TimeDistributed(Activation('tanh')))
# model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
# model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
# model.add(TimeDistributed(Activation('tanh')))
# model.add(TimeDistributed(Dropout(0.2093)))
# model.add(TimeDistributed(Activation('relu')))
# model.add(Flatten())
# model.add(Dense(512, activation='tanh'))
# model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
# model.add(Dense(n_outputs, activation='softmax'))